<a href="https://colab.research.google.com/github/Erickrus/leetcode/blob/master/run_squad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# download bert source codes
!mkdir -p /content/drive/My\ Drive/workspace
!cd /content/drive/My\ Drive/workspace && git clone https://github.com/google-research/bert

# download squad
!mkdir -p /content/drive/My\ Drive/workspace/squad
!cd /content/drive/My\ Drive/workspace/squad && wget -c https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
!cd /content/drive/My\ Drive/workspace/squad && wget -c https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json

# download bert model
!rm -Rf  /content/drive/My\ Drive/workspace/bert_model
!mkdir -p /content/drive/My\ Drive/workspace/bert_model
!cd /content/drive/My\ Drive/workspace/bert_model && wget -c https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!cd /content/drive/My\ Drive/workspace/bert_model && unzip uncased_L-12_H-768_A-12.zip -d /content/drive/My\ Drive/workspace/bert_model

!mv /content/drive/My\ Drive/workspace/bert_model/uncased_L-12_H-768_A-12/* /content/drive/My\ Drive/workspace/bert_model
!rmdir  /content/drive/My\ Drive/workspace/bert_model/uncased_L-12_H-768_A-12
!rm /content/drive/My\ Drive/workspace/bert_model/uncased_L-12_H-768_A-12.zip
!ls -al /content/drive/My\ Drive/workspace/bert_model

# prepare running path
!mkdir -p /content/drive/My\ Drive/workspace/bert_squad

In [0]:
import json
import os

# minify the squad dev file to a smaller size
def minify_squad_dev_file(
    datasetNum=1,
    paragraphNum=1, 
    qasNum=1):
  squadDir = "/content/drive/My Drive/workspace/squad"
  squadDevFilename = "dev-v1.1.json"
  squadDevMinFilename = "dev-v1.1.min.json"
  
  squadObj = json.load(open(os.path.join(squadDir, squadDevFilename)))
  
  squadObj["data"] = squadObj["data"][0:datasetNum]
  for k in range(datasetNum):
    squadObj["data"][k]["paragraphs"] = squadObj["data"][k]["paragraphs"][0:paragraphNum]
    for i in range(paragraphNum):
      squadObj["data"][k]["paragraphs"][i]["qas"] = squadObj["data"][k]["paragraphs"][i]["qas"][0:qasNum]
      for j in range(qasNum):
        squadObj["data"][k]["paragraphs"][i]["qas"][j]["answers"] = squadObj["data"][k]["paragraphs"][i]["qas"][j]["answers"][0:1]

  json.dump(
    squadObj, 
    open(os.path.join(squadDir, squadDevMinFilename), 'w'),
    indent=2, 
    sort_keys=True
  )
  
minify_squad_dev_file(2,2,2)
!cat /content/drive/My\ Drive/workspace/squad/dev-v1.1.min.json

In [72]:
def create_runnable():
  shContent = """
  cd /content/drive/My\ Drive/workspace
  
  WORKSPACE=`pwd`
  cd bert
  
  python3 -u run_squad.py \
  --bert_model bert-base-uncased \
  --do_train \
  --do predict \
  --do_lower_case \
  --train_file "$WORKSPACE/squad/dev-v1.1.min.json" \
  --predict_file "$WORKSPACE/squad/dev-v1.1.min.json" \
  --train_batch_size 8 \
  --learning_rate 3e-5 \
  --num_train_epochs 2.0 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --max_steps 4 \
  --vocab_file "$WORKSPACE/vocab.txt" \
  --bert_config_file "$WORKSPACE/bert_config.json" \
  --output_dir "$WORKSPACE/bert_squad"
  
  """
  bertDir = "/content/drive/My Drive/workspace/bert"
  shFile = open(os.path.join(bertDir, "run.sh"), "w")
  shFile.write(shContent)
  shFile.close()

create_runnable()
!cd /content/drive/My\ Drive/workspace/bert && cat run.sh && chmod 777 run.sh



  cd /content/drive/My\ Drive/workspace
  #cp -Rf /content/drive/My\ Drive/workspace/bert/* /content
  #cp -Rf /content/drive/My\ Drive/workspace/squad/*.json /content
  #mkdir /content/bert_squad
  WORKSPACE=`pwd`
  cd bert
  
  python3 -u run_squad.py   --bert_model bert-base-uncased   --do_train   --do predict   --do_lower_case   --train_file "$WORKSPACE/squad/dev-v1.1.min.json"   --predict_file "$WORKSPACE/squad/dev-v1.1.min.json"   --train_batch_size 8   --learning_rate 3e-5   --num_train_epochs 2.0   --max_seq_length 384   --doc_stride 128   --max_steps 4   --vocab_file "$WORKSPACE/vocab.txt"   --bert_config_file "$WORKSPACE/bert_config.json"   --output_dir "$WORKSPACE/bert_squad"
  
  

In [83]:
!cd /content/drive/My\ Drive/workspace/bert && ./run.sh



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Using config: {'_model_dir': '/content/drive/My Drive/workspace/bert_squad', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f573aec6710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evalua